# Complementaria Repaso Quiz 3 - Miércoles 12:30

In [1]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp

In [2]:
# Matrices de transición (orden de estados: T, N)

P_A = np.array([
    [0.75, 0.25],
    [0.60, 0.40]
])

P_C = np.array([
    [0.55, 0.45],
    [0.50, 0.50]
])

P_K = np.array([
    [0.80, 0.20],
    [0.55, 0.45]
])

P_D = np.array([
    [0.55, 0.45],
    [0.10, 0.90]
])

# Ingresos por estado (USD por mes)
ingresos = {
    'T': 140_000,
    'N': 40_000
}

# Costos por acción (USD por mes)
costos = {
    'A': -20_000,   # Nuevo álbum
    'C': -10_000,   # Colaboración
    'K': -25_000,   # Concierto
    'D': 0          # Descanso
}

## Parte A: Implementación del MDP

### 1. Crear el MDP en jmarkov

In [3]:
# Factor de descuento
beta = 0.95

# Espacio de estados y acciones
# T: Trending, N: No Trending
estados = np.array(['T', 'N'], dtype=str)

# A: Nuevo álbum, C: Colaboración, K: Concierto, D: Descanso
acciones = np.array(['A', 'C', 'K', 'D'], dtype=str)

# Diccionario de matrices de transición por acción
matrices = {}
for a in acciones:
    if a == 'A':
        matrices[a] = P_A
    elif a == 'C':
        matrices[a] = P_C
    elif a == 'K':
        matrices[a] = P_K
    else:  # 'D'
        matrices[a] = P_D

# Matriz de retornos R(s,a) = ingreso(s) + costo(a)
# filas: estados [T, N], columnas: acciones [A, C, K, D]
retornos = np.zeros((len(estados), len(acciones)), dtype=float)

for i, s in enumerate(estados):
    for j, a in enumerate(acciones):
        retornos[i, j] = ingresos[s] + costos[a]

retornos_df = pd.DataFrame(retornos, index=estados, columns=acciones)
retornos_df


,A,C,K,D
T,120000.0,130000.0,115000.0,140000.0
N,20000.0,30000.0,15000.0,40000.0


In [4]:
# Crear objeto MDP
mdp = dtmdp(estados, acciones, matrices, retornos, beta)
mdp

### 2. Valores óptimos usando iteración de valores (maximización)

In [5]:
value_functions, optimal_policy = mdp.solve(0, minimize=False, method='value_iteration')

In [6]:
# Política óptima
optimal_policy

{np.str_('T'): np.str_('K'), np.str_('N'): np.str_('A')}

In [7]:
# Valores óptimos por estado
value_functions

array([1854320.98765414, 1737037.03703685])

### 3. Valor esperado de la política óptima (largo plazo)

In [8]:
mdp.expected_policy_value(value_functions, optimal_policy)

np.float64(1824999.9999998142)

### 4. Matriz de transición inducida por la política óptima

In [9]:
matrizPO = mdp.policy_transition_matrix(optimal_policy)

# En DataFrame para indexar por letras de estado
matrizPO_df = pd.DataFrame(matrizPO, index=estados, columns=estados)
matrizPO_df

,T,N
T,0.8,0.2
N,0.6,0.4


## Parte B: Simulación de Monte Carlo

### Forma separada

In [10]:
escenarios = 1000
meses = 20

np.random.seed(0)

# Matriz para guardar la ganancia de cada mes y escenario
ganancia = np.zeros((meses, escenarios), dtype=float)

for j in range(escenarios):
    
    # Estado inicial: No Trending (N)
    estado = 'N'
    
    for i in range(meses):
        # Acción según política óptima
        accion = str(optimal_policy[estado])
        
        # Retorno inmediato en ese mes
        ganancia[i, j] = retornos_df.loc[estado, accion]
        
        # Estado futuro según matriz de transición inducida por la politica óptima
        estado = np.random.choice(estados, p=matrizPO_df.loc[estado, :])


In [11]:
# Últimos 5 meses: índices -5 a -1
ultimos_5 = ganancia[-5:, :]
total_ultimos_5_por_escenario = ultimos_5.sum(axis=0)
ingreso_esp_ultimos_5 = total_ultimos_5_por_escenario.mean()

ingreso_esp_ultimos_5


np.float64(459100.0)

In [12]:
escenarios = 1000
meses = 20

np.random.seed(0)

# Matriz para marcar 1 si está en T, 0 si no (para la segunda pregunta)
indicador_T = np.zeros((meses, escenarios), dtype=float)

for j in range(escenarios):
    
    # Estado inicial: No Trending (N)
    estado = 'N'
    
    for i in range(meses):
        # Acción según política óptima
        accion = str(optimal_policy[estado])
        
        # Marcar si está en Trending (T)
        if estado == 'T':
            indicador_T[i, j] = 1
        
        # Estado futuro según matriz de transición inducida por la politica óptima
        estado = np.random.choice(estados, p=matrizPO_df.loc[estado, :])


In [13]:
# Porcentaje de meses en los que BadDoggy está Trending (T)
porcentaje_T = indicador_T.mean() * 100
porcentaje_T


np.float64(70.62)

### Forma conjunta

In [14]:
escenarios = 1000
meses = 20

np.random.seed(0)

# Matriz para guardar la ganancia de cada mes y escenario
ganancia = np.zeros((meses, escenarios), dtype=float)

# Matriz para marcar 1 si está en T, 0 si no (para la segunda pregunta)
indicador_T = np.zeros((meses, escenarios), dtype=float)

for j in range(escenarios):
    
    # Estado inicial: No Trending (N)
    estado = 'N'
    
    for i in range(meses):
        # Acción según política óptima
        accion = str(optimal_policy[estado])
        
        # Retorno inmediato en ese mes
        ganancia[i, j] = retornos_df.loc[estado, accion]
        
        # Marcar si está en Trending (T)
        if estado == 'T':
            indicador_T[i, j] = 1
        
        # Estado futuro según matriz de transición inducida por la politica óptima
        estado = np.random.choice(estados, p=matrizPO_df.loc[estado, :])


In [15]:
# Últimos 5 meses: índices -5 a -1
ultimos_5 = ganancia[-5:, :]
total_ultimos_5_por_escenario = ultimos_5.sum(axis=0)
ingreso_esp_ultimos_5 = total_ultimos_5_por_escenario.mean()

ingreso_esp_ultimos_5


np.float64(459100.0)

In [16]:
# Porcentaje de meses en los que BadDoggy está Trending (T)
porcentaje_T = indicador_T.mean() * 100
porcentaje_T


np.float64(70.62)